In [ ]:
import torch
import cv2
import numpy as np

def predict_mask(model, image, device):
    """
    Runs inference on a new image and returns the predicted segmentation mask.
    """
    model.eval()
    image = image.to(device).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)  # Model prediction

    predicted_mask = torch.argmax(output, dim=1).squeeze(0).cpu().numpy()  # Convert to NumPy
    return predicted_mask


In [12]:
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp

# 🔹 Load the Model with its original structure
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = smp.DeepLabV3Plus(
    encoder_name="efficientnet-b4",
    encoder_weights=None,  # We are loading our trained weights
    in_channels=3,
    classes=2
)

# 🔹 Load weights (no modifications yet)
checkpoint = torch.load("best_model_effnet_deepl.pth", map_location=device)
model.load_state_dict(checkpoint, strict=False)

# 🔹 Now modify the segmentation head AFTER loading weights
model.segmentation_head = nn.Sequential(
    nn.Dropout(0.3),  # 30% dropout
    model.segmentation_head  # Keep the original segmentation head
)

# Move model to device
model.to(device)
model.eval()

print("✅ Model successfully loaded with modified segmentation head!")


✅ Model successfully loaded with modified segmentation head!


In [ ]:
import torch
import cv2
import numpy as np
import os
import pandas as pd
from torchvision import transforms
from PIL import Image
import segmentation_models_pytorch as smp

# 🔹 Define Paths
image_dir = "/Users/joaop.cardoso/MestradoCD/CAA/Project 1/images"
test_csv = "Train.csv"
output_dir = "output_images"

# 🔹 Image Preprocessing (Matches Training Pipeline)
transform = transforms.Compose([
    transforms.Resize((512, 512)),  # Resize to match model input
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

# 🔹 Prediction Function
def predict_mask(model, image, device, original_size):
    image = image.to(device).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)  # Model prediction

    predicted_mask = torch.argmax(output, dim=1).squeeze(0).cpu().numpy()  # Convert to NumPy

    # 🔹 Resize mask back to original image size
    predicted_mask = cv2.resize(predicted_mask, (original_size[1], original_size[0]), interpolation=cv2.INTER_NEAREST)

    return predicted_mask

# 🔹 Read Test Image IDs
df_test = pd.read_csv(test_csv)

# 🔹 Process Each Image in Test.csv
for index, row in df_test.iterrows():
    image_id = row["ID"]  # Extract image ID
    
    image_path = os.path.join(image_dir, f"{image_id}.jpg")
    if not os.path.exists(image_path):
        print(f"❌ Image {image_id}.jpg not found, skipping.")
        continue

    # Load and preprocess image
    original_image = Image.open(image_path).convert("RGB")
    original_size = original_image.size[::-1]  # (height, width)
    
    image = transform(original_image)  # Apply transformations

    # Get Predictions
    predicted_mask = predict_mask(model, image, device, original_size)

    # Convert to BGR image format for OpenCV
    original_image = np.array(original_image)

    # Create an RGB mask overlay
    overlay = np.zeros_like(original_image, dtype=np.uint8)

    # Apply colors based on predicted mask
    overlay[predicted_mask == 0] = (0, 165, 255)  # Orange (Boiler)
    overlay[predicted_mask == 1] = (0, 255, 255)  # Yellow (Solar Panel)

    # Blend the overlay with the original image
    blended_image = cv2.addWeighted(original_image, 0.6, overlay, 0.4, 0)

    # Save the resulting image
    output_path = os.path.join(output_dir, f"{image_id}_pred.png")
    cv2.imwrite(output_path, cv2.cvtColor(blended_image, cv2.COLOR_RGB2BGR))

    print(f"✅ Saved: {output_path}")

print(f"\n🎉 Processing complete! All predicted images are saved in `{output_dir}`.")


✅ Saved: output_images/ID00rw8_pred.png
✅ Saved: output_images/ID014O6EC7_pred.png
✅ Saved: output_images/ID020cu0z_pred.png
✅ Saved: output_images/ID024YTBkLvRpQahT_pred.png
✅ Saved: output_images/ID024YTBkLvRpQahT_pred.png
✅ Saved: output_images/ID024YTBkLvRpQahT_pred.png
✅ Saved: output_images/ID02vByTw8Htl_pred.png
✅ Saved: output_images/ID04S4rM9Qs_pred.png
✅ Saved: output_images/ID04S4rM9Qs_pred.png
✅ Saved: output_images/ID06Zb8HWf7XPxodI_pred.png
✅ Saved: output_images/ID08l4qXO27c08_pred.png
✅ Saved: output_images/ID08lsWcpi7_pred.png
✅ Saved: output_images/ID09o6w2iKE7RG_pred.png
✅ Saved: output_images/ID09pzWgf_pred.png
✅ Saved: output_images/ID09pzWgf_pred.png
✅ Saved: output_images/ID09pzWgf_pred.png
✅ Saved: output_images/ID09pzWgf_pred.png
✅ Saved: output_images/ID09pzWgf_pred.png
✅ Saved: output_images/ID09pzWgf_pred.png


KeyboardInterrupt: 